In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')

In [3]:
# Manage paths
%run setup_notebooks.py

In [4]:
from toolbox import wandb_export
from src import setup

In [5]:
import logging
logging.getLogger().setLevel(logging.INFO)

---

In [6]:
path = setup.get_dataset_path('models')

2022-04-20 14:10:58,915 [setup] [INFO] get_data_path(): using user-defined data path


In [8]:
import wandb

In [9]:
runs = wandb.Api().runs('mazeller/egnn-mi', filters={'tags': 'test-ggho5n'})
len(runs)

5

In [10]:
ids = [e.id for e in runs]
ids

['2twydzvi', '1nk4mviw', '13ydacsh', '1jm36hca', '2vu548g3']

In [19]:
df = []
for i in ids:
    sub = pd.read_json(path.joinpath(f'pred-{i}.json'))
    sub['id'] = i
    df.append(sub)
    
df = pd.concat(df, ignore_index=True)
df

,file,type,g_x,pred,y,id
0,CHUV01_LAD.pt,train,"[[1.0, 0.0, 0.0]]","[0.36181354522705, 0.638186514377594]",0,2twydzvi
1,CHUV01_LAD_diffusion_0.pt,train,"[[1.0, 0.0, 0.0]]","[0.278908342123031, 0.7210916876792901]",0,2twydzvi
2,CHUV01_LAD_diffusion_1.pt,train,"[[1.0, 0.0, 0.0]]","[0.5399911403656, 0.460008889436721]",0,2twydzvi
3,CHUV01_LCX.pt,train,"[[0.0, 1.0, 0.0]]","[0.9981886744499201, 0.0018113497644660001]",1,2twydzvi
4,CHUV01_LCX_diffusion_0.pt,train,"[[0.0, 1.0, 0.0]]","[0.9994127750396721, 0.000587254122365]",1,2twydzvi
...,...,...,...,...,...,...
2287,OLV041_LAD.pt,test,"[[1.0, 0.0, 0.0]]","[0.690310060977935, 0.30968990921974104]",0,2vu548g3
2288,OLV041_LCX.pt,test,"[[0.0, 1.0, 0.0]]","[0.972227692604064, 0.027772255241870002]",0,2vu548g3
2289,OLV041_RCA.pt,test,"[[0.0, 0.0, 1.0]]","[0.798999309539794, 0.20100072026252702]",1,2vu548g3
2290,OLV048_LAD.pt,test,"[[1.0, 0.0, 0.0]]","[0.33530977368354803, 0.6646901965141291]",1,2vu548g3


In [20]:
df['yhat'] = df.pred.apply(np.argmax)
df['correct'] = df.y == df.yhat
df['aug'] = df.file.str.contains('diffusion')

In [23]:
train = df[df.type == 'train']
train.groupby(['id', 'aug']).correct.mean()

id        aug  
13ydacsh  False    0.637037
          True     0.651852
1jm36hca  False    0.607407
          True     0.655556
1nk4mviw  False    0.601449
          True     0.659420
2twydzvi  False    0.644444
          True     0.711111
2vu548g3  False    0.601504
          True     0.642857
Name: correct, dtype: float64